In [1]:
%load_ext autoreload
%autoreload 2

import os
import json
import copy
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.linear_model import LinearRegression
import ray_results_interpreter as rri
import subprocess
import concurrent.futures
from main_run import MainRun

In [2]:
# Create vanilla results dataframe from the provided data
testset_name = "finals_serial_HDPO"

results_interpretor = rri.RayResultsinterpreter()

def custom_data_filler(out_row, reference_row):
    out_row['path'] = reference_row['path']

def default_condition_setter(condition_name):
    return None

architectures = {
    "Vanilla NN": lambda config: f'/user/ml4723/Prj/NIC/ray_results/{testset_name}/vanilla_serial',
    "echelon_stock_hard": lambda config: f'/user/ml4723/Prj/NIC/ray_results/{testset_name}/echelon_stock_hard',
}

sort_by = 'dev_loss'
pick_row_from_run_by = 'dev_loss'

config = "one_store_backlogged"
store_lead_times = [1, 2, 3, 4]
store_underage_costs = [4, 9, 19, 39]

dfs = []

for arch_name, path_fn in architectures.items():
    path = path_fn(config)

    df = results_interpretor.make_table({1: path},
        {'store_underage_cost': store_underage_costs,
            'store_lead_time': store_lead_times},
        default_condition_setter, custom_data_filler,
        sort_by=sort_by, pick_row_from_run_by=pick_row_from_run_by, test_loss_limit=25)
    if df.empty:
        continue

    df.insert(2, 'Architecture Class', arch_name)
    df.insert(1, 'hyperparam_name', arch_name)
    df['config'] = config
    dfs.append(df)

if not dfs:
    raise ValueError("No dataframes found for the given settings.")

df = pd.concat(dfs, ignore_index=True)

In [3]:
import numpy as np

test_loss_column = 'Test Loss'
test_loss_filename = 'serial_system_4_test_loss.txt'

test_losses = []
for _, row in df.iterrows():
    test_loss_path = str(row['path']) + "/" + test_loss_filename
    try:
        with open(test_loss_path, 'r') as f:
            value = float(f.read().strip())
    except Exception:
        value = np.nan
    test_losses.append(value)

df[test_loss_column] = test_losses

In [ ]:
import numpy as np
import pandas as pd
import os

lead_time_col = "store_lead_time"
underage_cost_col = "store_underage_cost"
dev_loss_col = "Dev Loss"
train_loss_col = "Train Loss"
hyperparam_col = "hyperparam_name"
echelon_stock_hard_hyperparam = "echelon_stock_hard"
test_loss_col = "Test Loss"

# Only consider vanilla NN runs (not base_stock)
vanilla_df = df[df[hyperparam_col] != "echelon_stock_hard"].copy()

# Get optimal losses from echelon_stock_hard runs
echelon_df = df[df[hyperparam_col] == echelon_stock_hard_hyperparam].copy()
optimal_loss_dict = {}
for _, row in echelon_df.iterrows():
    lead_time = int(row[lead_time_col])
    underage_cost = int(row[underage_cost_col])
    test_loss = row.get(test_loss_col, np.nan)
    if not np.isnan(test_loss):
        optimal_loss_dict[(underage_cost, lead_time)] = test_loss

rows = []
for _, row in vanilla_df.iterrows():
    lead_time = int(row[lead_time_col])
    underage_cost = int(row[underage_cost_col])
    optimal_loss = optimal_loss_dict.get((underage_cost, lead_time), np.nan)
    if np.isnan(optimal_loss):
        continue

    train_loss = row.get(train_loss_col, np.nan)
    dev_loss = row.get(dev_loss_col, np.nan)
    test_loss = row.get(test_loss_col, np.nan)
    if np.isnan(test_loss):
        continue

    test_gap = 100 * (test_loss - optimal_loss) / optimal_loss
    test_gap_str = "<0.25" if test_gap < 0.25 else f"{test_gap:.2f}"

    # Compute train gap and dev gap as in the table
    train_gap = 100 * (train_loss - optimal_loss) / optimal_loss if not np.isnan(train_loss) else np.nan
    dev_gap = 100 * (dev_loss - optimal_loss) / optimal_loss if not np.isnan(dev_loss) else np.nan
    train_gap_str = "<0.25" if not np.isnan(train_gap) and train_gap < 0.25 else (f"{train_gap:.2f}" if not np.isnan(train_gap) else "")
    dev_gap_str = "<0.25" if not np.isnan(dev_gap) and dev_gap < 0.25 else (f"{dev_gap:.2f}" if not np.isnan(dev_gap) else "")

    progress_path = os.path.join(row["path"], "progress.csv")
    grad_steps_to_1pct = ""
    time_to_1pct = ""
    try:
        progress_df = pd.read_csv(progress_path)
        dev_gap_threshold = 0.01 * optimal_loss
        first_time = progress_df["time_total_s"].iloc[0]
        first_step = progress_df["training_iteration"].iloc[0]
        found = False
        for idx, prog_row in progress_df.iterrows():
            prog_dev_gap = abs(prog_row["dev_loss"] - optimal_loss)
            if prog_dev_gap <= dev_gap_threshold:
                grad_steps = prog_row["training_iteration"] - first_step
                time_to_1pct = prog_row["time_total_s"] - first_time
                grad_steps_scaled = grad_steps * 10 * 4  # 10x for epoch, 32x for batch size 1024
                grad_steps_to_1pct = int(grad_steps_scaled)
                time_to_1pct = int(time_to_1pct)
                found = True
                break
        if not found:
            grad_steps_to_1pct = ""
            time_to_1pct = ""
    except Exception:
        grad_steps_to_1pct = ""
        time_to_1pct = ""

    rows.append([
        lead_time,
        underage_cost,
        round(train_loss, 2) if not np.isnan(train_loss) else "",
        round(dev_loss, 2) if not np.isnan(dev_loss) else "",
        round(test_loss, 2),
        train_gap_str,
        dev_gap_str,
        test_gap_str,
        grad_steps_to_1pct,
        time_to_1pct
    ])

table9_df = pd.DataFrame(rows, columns=[
    "Store lead time",
    "Store underage cost",
    "Train loss",
    "Dev loss",
    "Test loss",
    "Train gap (%)",
    "Dev gap (%)",
    "Test gap (%)",
    "Gradient steps to 1% dev gap",
    "Time to 1% dev gap (s)"
])

table9_df = table9_df.sort_values(["Store lead time", "Store underage cost"]).reset_index(drop=True)
table9_df

,Store lead time,Store underage cost,Train loss,Dev loss,Test loss,Train gap (%),Dev gap (%),Test gap (%),Gradient steps to 1% dev gap,Time to 1% dev gap (s)
0,1,4,6.94,6.94,6.93,0.59,0.53,0.50,4960,1414
1,1,9,8.42,8.41,8.41,0.66,0.49,0.48,4320,1255
2,1,19,9.79,9.81,9.80,1.93,2.09,2.04,,
3,1,39,10.96,10.95,10.95,2.46,2.37,2.32,,
4,2,4,7.63,7.64,7.64,0.43,0.53,0.46,7200,2185
5,2,9,9.31,9.33,9.32,0.49,0.73,0.68,4680,1413
6,2,19,10.73,10.75,10.75,0.56,0.77,0.81,4480,1389
7,2,39,11.96,11.96,11.97,0.45,0.45,0.52,4040,1251
8,3,4,8.28,8.27,8.26,0.84,0.74,0.65,5080,1419
9,3,9,10.11,10.11,10.11,0.67,0.70,0.66,5320,1623


In [4]:
mode = "test"
setting_names = ['serial_system_4']

models = []
for _, row in df.iterrows():
    models.append(str(row['path']) + '/model.pt')

gpus = [0, 1, 2, 3]

import nest_asyncio
import asyncio

nest_asyncio.apply()

async def run_main_run(model_path, setting_name, gpu_idx, semaphore):
    async with semaphore:
        try:
            hyperparam_name = model_path.split('/')[7]
            print(f"Running main_run.py for path {model_path}")
            cmd = [
                "/user/ml4723/.conda/envs/neural_inventory_control/bin/python",
                "main_run.py",
                mode,
                setting_name,
                hyperparam_name,
                str(model_path),
                str(gpus[gpu_idx])
            ]
            env = {
                **os.environ,
                "MKL_THREADING_LAYER": "GNU",
                "MKL_SERVICE_FORCE_INTEL": "1"
            }
            process = await asyncio.create_subprocess_exec(
                *cmd,
                env=env,
                cwd="/user/ml4723/Prj/NIC/",
                stdout=asyncio.subprocess.PIPE,
                stderr=asyncio.subprocess.PIPE
            )
            stdout, stderr = await process.communicate()
            if process.returncode != 0:
                print(f"Error running main_run.py for path {model_path}: {stderr.decode()}")
        except Exception as e:
            print(f"Unexpected error running main_run.py for path {model_path}: {e}")

async def main():
    max_concurrent = 6 * len(gpus)
    semaphore = asyncio.Semaphore(max_concurrent)
    tasks = []
    gpu_idx = 0
    for setting_name in setting_names:
        for path in models:
            tasks.append(run_main_run(path, setting_name, gpu_idx, semaphore))
            gpu_idx = (gpu_idx + 1) % len(gpus)
    await asyncio.gather(*tasks)

await main()


Running main_run.py for path /Users/minuk.lee/Downloads/Neural_inventory_control/ray_results/finals_serial_HDPO/vanilla_serial/run_2025-05-23_16-59-36/run_d57a0_00000_0_config=serial_system_4,early_stop_check_epochs=10,repeats=1,stop_if_no_improve_for_epochs=500,store_lead_time=1,_2025-05-23_16-59-36/model.pt
Running main_run.py for path /Users/minuk.lee/Downloads/Neural_inventory_control/ray_results/finals_serial_HDPO/vanilla_serial/run_2025-05-23_16-59-36/run_d57a0_00001_1_config=serial_system_4,early_stop_check_epochs=10,repeats=1,stop_if_no_improve_for_epochs=500,store_lead_time=2,_2025-05-23_16-59-36/model.pt
Running main_run.py for path /Users/minuk.lee/Downloads/Neural_inventory_control/ray_results/finals_serial_HDPO/vanilla_serial/run_2025-05-23_16-59-36/run_d57a0_00002_2_config=serial_system_4,early_stop_check_epochs=10,repeats=1,stop_if_no_improve_for_epochs=500,store_lead_time=3,_2025-05-23_16-59-36/model.pt
Running main_run.py for path /Users/minuk.lee/Downloads/Neural_inve

Running main_run.py for path /Users/minuk.lee/Downloads/Neural_inventory_control/ray_results/finals_serial_HDPO/echelon_stock_hard/run_2025-05-23_19-22-50/run_d871e_00074_74_config=serial_system_4,early_stop_check_epochs=10,learning_rate=0.0300,repeats=1,stop_if_no_improve_for_epochs=_2025-05-23_19-22-56/model.pt
Running main_run.py for path /Users/minuk.lee/Downloads/Neural_inventory_control/ray_results/finals_serial_HDPO/echelon_stock_hard/run_2025-05-23_19-22-50/run_d871e_00081_81_config=serial_system_4,early_stop_check_epochs=10,learning_rate=0.5000,repeats=1,stop_if_no_improve_for_epochs=_2025-05-23_19-22-56/model.pt
Running main_run.py for path /Users/minuk.lee/Downloads/Neural_inventory_control/ray_results/finals_serial_HDPO/echelon_stock_hard/run_2025-05-23_19-22-50/run_d871e_00090_90_config=serial_system_4,early_stop_check_epochs=10,learning_rate=0.5000,repeats=1,stop_if_no_improve_for_epochs=_2025-05-23_19-22-56/model.pt
Running main_run.py for path /Users/minuk.lee/Downloads